# Bigram Language Model 

This is based on Andrej's Youtube video [The spelled-out intro to language modeling: building makemore](https://www.youtube.com/watch?v=PaCmpygFfXo).

It explains how to develop a simple bigram language model using a neural network. It covers model training, sampling, and the evaluation of a loss function.

- [bigram notebook file](https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part1_bigrams.ipynb)
- [makemore repository](https://github.com/karpathy/makemore)

## The Dataset and Language Model

In [84]:
words = open("names.txt", "r").read().splitlines()

In [ ]:
print(words[:10])
print(len(words))

lengths = [len(word) for word in words]
print(min(lengths), max(lengths))

### Character-Level Language Model

A character-level language tries to predict the next character based on the preceding characters. There are `32033` words that each word has a length from `2` to `15`.

For example, in the word `isabella`:

- `i` is the first character.
- each middle character follows another character.
- after the last `a`, the word terminates.

### Bigram Language Model

A bigram language model works on two characters at a time.

It only looks one character and predictes the next character.

It is a simple language model for pedagogical purpose. 

### Bigram From Words

Each word generates `n-1` pairs of characters, where `n` is the length of the word

In [ ]:
# each word generates n-1 pairs of characters
# where n is the length of the word
for w in words[:3]:
    for ch1, ch2 in zip(w, w[1:]):
        print(ch1, ch2)

### Start and End

However, there are two implict characters in each word: `<S>` and `<E>`.

Thus the code to generating bigrams is as below:

In [ ]:
for w in words[:3]:
    chars = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chars, chars[1:]):
        print(ch1, ch2)

### Bigram Frequency

The prediction is based on the bigram frequencies.


In [88]:
b = {}
for w in words:
    chars = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chars, chars[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [ ]:
iterator = iter(b.items())
three = [next(iterator) for _ in range(3)]
print(list(three))
sorted(b.items(), key=lambda item: item[1], reverse=True)[:3]

In [90]:
import torch

In [ ]:
a = torch.zeros((3, 5), dtype=torch.int32)
a[0, 0] = 5
a[1, 3] += 1
a

In [92]:
# convert characters to integers
chars = sorted(list(set("".join(words))))
# print(chars)
stoi = {ch: i + 1 for i, ch in enumerate(chars)}
stoi["."] = 0

itos = {i: ch for ch, i in stoi.items()}

In [93]:
N = torch.zeros((27, 27), dtype=torch.int32)

for w in words:
    chars = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chars, chars[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

In [95]:
# for each row, create a probability distribution
# P = N.float()
P = (N + 1).float()
P = P / P.sum(dim=1, keepdim=True)  # normalize each row

In [ ]:
P[0].sum()

### Broadcasting Semantics

The calculation `P = P / P.sum(dim=1, keepdim=True)` is a division between two matrix: `[27, 27] / [27, 1]`. It follows PyTorch [broadcasting semantics](https://pytorch.org/docs/stable/notes/broadcasting.html).

- When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is `1`, or one of them does not exist.
- If the number of dimensions of `x` and `y` are not equal, prepend `1` to the dimensions of the tensor with fewer dimensions to make them equal length.
- Then, for each dimension size, the resulting dimension size is the max of the sizes of `x` and `y` along that dimension.

In the above, the two `broadcastable` based on the above rules. The column vector `[27, 1]` is extended by copying the first column `27` times to match the size of the first matrix. Then it does an element-wise division.

### Broadcasting Can Cause Bug

In the above code, if the second argument of  `P.sum(dim=1, keepdim=True)` is missing, the result is a `[27]` vector. In division, it is treated as `[1, 27]`. the values will be replicated `27` times for each row to become `[27, 27]`.

The correct one should replicate the `27` sum values for each column. Use a three value `(10, 20, 30)` example, each value is the sum of each row.

- Missing `keepdim=True`, the shape is `[27]` and values are replicated by rows (vertically): `[[10, 20, 30], [10, 20, 30], [10, 20, 30]]`.
- With `keepdim=True`, the shape is `[27, 1]` and  values are replicated by column (horizontally): `[[10, 10, 10], [20, 20, 20], [30, 30, 30]]`.

Because each value `(10, 20, 30)` in is the sum of each row, every element in the first row should be divided by `10`, every element in the second row should be divided by `20`, an so on so forth. Therefore, `keepdim=True` gives the correct result.

In [ ]:
# use generator to ensure reproducibility in the same computer !
g = torch.Generator().manual_seed(2147483647)
for _ in range(10):
    out = []
    ix = 0
    while True:
        ix = torch.multinomial(P[ix], 1, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break

    print("".join(out))

### The Model Quality

Using [Maximum likelihood estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation), the likelihood is the product of probabilities of each pair. The likelihood of the entire data set is the measured quality of the training model. Higher is better.

The real probability for each pair is typically small, for example, the average should be `1/27 = 0.037`.  Thus, `logprob` is used. The range of `logprob` is from `-infinity` (when `p = 0`) to `0` (when `p=1`). The likelihood for `logprob` is the sum of the `logprob`.

To make it a loss function (smaller is btter), use `nll` (negative log likelihood) that is the average negative log likilihood of each pair. The lowest `nll` value is `0`.

Thus. to maximize the likelihood is to minimize the `nll`.

In [ ]:
n = 0
log_likelihood = 0

# for w in ["andrej"]:

# # this "jq" pair is not in the dataset, so the log likelihood should be -inf, need smoothing. add one to every possible pair.
# for w in [
#     "andrejq"
# ]:

for w in words[:]:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f"{ch1}{ch2}: {prob:.4f}, {logprob:.4f}")

print(f"log likelihood: {log_likelihood:.4f}")
nll = -log_likelihood / n
print(f"nll: {nll:.4f}")

### So Far

The above code calculates normalized probability of every pair.

The pair probability can be used to generate new words using multinomial distribution.

`nll` is developed as the loss funciton to measure the model quality.

## NN Approach

The NN model receives a single character as an input and create an output that is a probability distribution over the next character.

The `nll` is used to tune the model parameters.

In [99]:
# create the training set of all bigrams (x, y).
xs, ys = [], []

for w in words[:1]:
    chars = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chars, chars[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)  # becarful the data type
ys = torch.tensor(ys)

In [ ]:
xs, ys

### Encoding

Neurons perform calculations on inputs with weights and biases. It doesn't make sense to use the character index as an input value.

PyTorch has `One Hot` encoding that creates vector for each index value, only the index poisition is `1`.

In [101]:
import torch.nn.functional as F

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()  # need to convert to float
print(xenc)
print(xenc.shape)
plt.imshow(xenc)

In [ ]:
xenc.shape

There are 27 different characters - that's the `num_classes` value is.
Each input is one of the 27 characters, the index position is 1, other positions are 0.
A single character in the data set is mapped into a tenor of the shape of `[27]`.

In [ ]:
W = torch.randn((27, 1))  # 1 neuron
print(W)
xenc @ W  # five inputs, one neuron, resul is 5x1

The neuron takes 27-dimensional inputs , corresponding to the `num_classes` of the one-hot input. Each character input is represented by a float tensor of shape `[27]`.

The `W` tensor is a NN layer where the first value is the dimension number of the input, the second is the number of neurons in the layer.

The neuron's weights are initilialized from standard normal distribution. Most values (about `68%`) are between `[-1, 1]`, few (about `0.3%`) is smaller than `-3` or larger than `3`.

In [ ]:
W = torch.randn((27, 27))  # 27 neurons
logits = xenc @ W  # five inputs, 27 neurons, result is [5, 27]

# matrix multiplication is the sum of dot product
print(logits[3, 13])
print((xenc[3] * W[:, 13]).sum())

The result of `xenc @ W` can be interpreted as the "firing rates" of neurons for the training samples. 

It is important to accurately interprete the 27 output values of the 27 neurons. Their initial values are **randomly** generated from standard normal distribution.

Intuitively, for each input, the NN predicts the probability distribution for the next character. There are 27 possible next characters. The sum of the probality is 1. 

So the goal is to transform the 27 output values into probability distribution.

### Softmax

Semantically, the output values of the one layer NN can be interpreted as a result of `log(counts)` - called count `logits`.

To get the counts, exponentiate the logit values. All values below `0` will be smaller than `1`. All positive numbers are transformed into values bigger than `1`. `logits.exp()` is similar to the `N` count matrix in the previous probability model.

Once we have the counts, probability distribution is easy. 

The whole process is a `softmax` function that "squashes" a K-dimensional float values to a K-dimensional vector of float values in the range `(0, 1)` that add up to 1 - the key feature of probability distribution.

In [ ]:
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)
probs[3].sum()

### The NN Model

- The input is `one-hot` encoded as a 27-dimensional data.
- Each neuron has a 27 inputs.
- A single layer has 27 neurons therefore it has 27 outputs.
- The neuron transformation is differentiable, therefore it supports backpropagation to update gradients and tune weights.
    - only linear transformation (even no bias)
    - the `softmax` function is also differentiable 
- The 27 outputs are interpreted the 27 probabilities for the 27 characters to be the next character for a given character.


### summary

- `xs` are inputs, `ys` are labels (the actual next characters).
- `g` is a generator that uses a manual seed thus the process is repeatable.
- `W` is an one-layer NN that have 27 neurons, each neuron receives 27 inputs.



In [123]:
xs, ys = [], []

for w in words[:1]:
    chars = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chars, chars[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)  # becarful the data type
ys = torch.tensor(ys)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)  # 27 neurons

### Foward Pass

The forward pass is differentiable: every operation can be back-backpropagated.

- predict the log-counts
- `softmax`

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()  # encode the input, convert to float
logits = xenc @ W  # predict the log-counts

# the softmax function
counts = logits.exp()  # counts, equivalent to N
probs = counts / counts.sum(dim=1, keepdim=True)  # probabilities, equivalent to P

### The Loss Function

For each input `i`,  `x = xs[i]` is the input character index, there is a corresponding label `y = ys[i]` where `y` is the index of the correct character.

Therefore, `probs[i, y]` is the predicted probability for `x`. Of course, `1` is a perfect probability value for `probs[i, y]`.

The loss function is defined as the mean of negative log likelihood of probabilities for all inputs.

In [129]:
num_inputs = len(xs)
nlls = torch.zeros(num_inputs)

for i in range(num_inputs):
    x = xs[i].item()
    y = ys[i].item()
    print(f"bigram: {itos[x]}{itos[y]}, index: {x}, {y}")
    nlls[i] = -torch.log(probs[i, y])
    print(f"prob: {probs[i, y]:.4f}, nll: {nlls[i]:.4f}")

print(f"average negative log likelihood, i.e. loss =  {nlls.mean().item():.4f}")

bigram: .e, index: 0, 5
prob: 0.0083, nll: 4.7944
bigram: em, index: 5, 13
prob: 0.0033, nll: 5.7115
bigram: mm, index: 13, 13
prob: 0.0181, nll: 4.0130
bigram: ma, index: 13, 1
prob: 0.0105, nll: 4.5541
bigram: a., index: 1, 0
prob: 0.0866, nll: 2.4464
average negative log likelihood, i.e. loss =  4.3039


### Different Loss Values

Different random intialization parameters generate different loss values.

The right approach is to tune `W` using gradient-based optimization - a typical NN training process. Repeat the following loop for a number of time; or the loss value is small enough; or there is no significant improvement.

- run forward pass
  - evaluate input
  - get loss value
- run backward pass
    - reset parameters
    - run `loss.backward()`
- update parameters
    - tune parameters with gradients and step size

## NN Advantages

The NN result is almost same as the best probability model - it proves its validity.

Scalable: more characters. 

Only change the neuron computation

Insights:

- In `xenc@W`, one hot encoding picks the specific row. `W` is log count, not raw count.
- Smoothing, increasing count smoot the probability. `W` is initialized randomly. If setting to 0, it smooths. can be used for regularization, used in loss funciton will push `W` to be 0. 